In [1]:
import numpy as np
import random 
import pandas as pd
import pyomo
import pyomo.environ as env
import matplotlib.pyplot as plt
import networkx as nx

from pyomo.environ import *
from pyomo.opt import SolverFactory

# 1. 

In [2]:
#Modelo
model = ConcreteModel()

#Conjuntos
distribuidores = {1,2}
compradores = {1,2,3,4,5,6}

#Parametros
ofertas =  {1:550, 2:700}
demandas = {1:125, 2:175, 3:225, 4:250, 5:225, 6:200}
costos = {
    (1,1):999,
    (1,2):2.5,
    (1,3):1.6,
    (1,4):1.4,
    (1,5):0.8,
    (1,6):1.4,
    (2,1):2.5,
    (2,2):999,
    (2,3):2.0,
    (2,4):1.0,
    (2,5):1.0,
    (2,6):0.8
}

#Variable de decision
model.x = Var(distribuidores, compradores, domain=NonNegativeReals)

#Funcion Objetivo
model.obj = Objective(
    expr = sum(sum(model.x[i,j]*costos[i,j] for j in compradores) for i in distribuidores), 
    sense = 1
)

#Restricciones
def con1(model, i):
    return sum(model.x[i,j] for j in compradores) <= ofertas[i]
model.con1 = Constraint(distribuidores, rule=con1)

def con2(model, j):
    return sum(model.x[i,j] for i in distribuidores) == demandas[j]
model.con2 = Constraint(compradores, rule=con2)

#Modelo
SolverFactory('glpk').solve(model)
model.display()

Model unknown

  Variables:
    x : Size=12, Index={1, 2}*{1, 2, 3, 4, 5, 6}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (1, 2) :     0 : 175.0 :  None : False : False : NonNegativeReals
        (1, 3) :     0 : 225.0 :  None : False : False : NonNegativeReals
        (1, 4) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (1, 5) :     0 : 150.0 :  None : False : False : NonNegativeReals
        (1, 6) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 1) :     0 : 125.0 :  None : False : False : NonNegativeReals
        (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 4) :     0 : 250.0 :  None : False : False : NonNegativeReals
        (2, 5) :     0 :  75.0 :  None : False : False : NonNegativeReals
        (2, 6) :     0 : 200.0 :  None : Fals

# 2.

In [10]:
#Modelo
model = ConcreteModel()

#Conjuntos
localidades = {0,1,2,3,4,5}

#Parametros
n = len(localidades)

df = pd.read_csv('./data2/proof_case.csv')
df.columns = [int(col.strip()) for col in df.columns]
df = df.apply(pd.to_numeric)
for i in range(len(df)):
    df.iloc[i, i] = 999

costos = {}
for origen in localidades:
    for destino in localidades:
        costos[(origen, destino)] = df[origen][destino]
        
#Variable de decision
model.x = Var(localidades, localidades, domain=Binary)
model.u = Var(localidades, domain=NonNegativeIntegers)

#Funcion Objetivo
model.obj = Objective(
    expr = sum(sum(model.x[i,j]*costos[i,j] for j in localidades) for i in localidades), 
    sense = 1
)

#Restricciones
def destiny_once(model, i):
    return sum(model.x[i,j] for j in localidades if j!=i) == 1
model.destiny_once = Constraint(localidades, rule=destiny_once)

def origin_once(model, j):
    return sum(model.x[i,j] for i in localidades if i!=j) == 1
model.origin_once = Constraint(localidades, rule=origin_once)

def sub_tour_elimination(model, i, j):
    if i!=j:
        return model.u[i] - model.u[j] + ((n-1) * model.x[i,j]) <= (n-2)
    else:
        return Constraint.Skip
model.sub_tour_elimination = Constraint(localidades, localidades, rule=sub_tour_elimination)

model.u_fixed = Constraint(expr=model.u[0] == 1)

#Modelo
SolverFactory('glpk').solve(model)
model.display()

Model unknown

  Variables:
    x : Size=36, Index={0, 1, 2, 3, 4, 5}*{0, 1, 2, 3, 4, 5}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :  None :     1 : False :  True : Binary
        (0, 1) :     0 :  None :     1 : False :  True : Binary
        (0, 2) :     0 :  None :     1 : False :  True : Binary
        (0, 3) :     0 :  None :     1 : False :  True : Binary
        (0, 4) :     0 :  None :     1 : False :  True : Binary
        (0, 5) :     0 :  None :     1 : False :  True : Binary
        (1, 0) :     0 :  None :     1 : False :  True : Binary
        (1, 1) :     0 :  None :     1 : False :  True : Binary
        (1, 2) :     0 :  None :     1 : False :  True : Binary
        (1, 3) :     0 :  None :     1 : False :  True : Binary
        (1, 4) :     0 :  None :     1 : False :  True : Binary
        (1, 5) :     0 :  None :     1 : False :  True : Binary
        (2, 0) :     0 :  None :     1 : False :  True : Binary
        (2, 1) 

# 3.